# 6b. Direct API Inspection of Profile Data

## Objective

This notebook takes a clean, direct approach to data exploration. Instead of pre-filtering, we will connect to the API and download the complete, national dataset for one sample indicator from each of our priority profiles. 

This allows us to inspect the raw data structure, including all available columns, before we decide on a filtering strategy. We will fetch data at the ICB level (`AreaTypeID = 153`) for a sample indicator from each of the 7 ranked profiles.

### 1. Setup and Library Imports

In [10]:
# %% Imports & display settings
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 120)

# %% Define your indicators
INDICATORS = {
    219: 'Hypertension QOF prevalence',
    241: 'Diabetes QOF prevalence',
    253: 'COPD QOF prevalence'
}

# %% Loop through each indicator
for ind_id, ind_name in INDICATORS.items():
    print(f"\n=== {ind_name} (ID: {ind_id}) ===")
    
    # 1) Fetch all available geographies for this indicator
    df_all = ftp.get_data_for_indicator_at_all_available_geographies(
        indicator_id=ind_id
    )
    
    if df_all.empty:
        print("⚠️  No data returned for any geography. Check the indicator ID.")
        continue
    
    # 2) Inspect the frame to find the geography-type column
    print("Columns detected:", df_all.columns.tolist())
    display(df_all.head())
    
    # Pick the correct geography column from likely candidates
    for col in ('AreaTypeID', 'AreaType', 'AreaTypeName'):
        if col in df_all.columns:
            geo_col = col
            break
    else:
        raise KeyError("Could not find any geography-type column in the DataFrame.")
    
    print(f"Using geography column: '{geo_col}'")
    unique_types = df_all[geo_col].unique()
    print("Available geography types:", unique_types)
    
    # 3) Filter for CCGs and GP practices
    targets = {
        'Clinical Commissioning Group': 'CCG',
        'GP practice':               'GP practice'
    }
    
    for type_name, label in targets.items():
        subset = df_all[df_all[geo_col] == type_name]
        if subset.empty:
            print(f" • No data for {label} (looking for '{type_name}').")
            continue
        
        print(f"\n📍 {label}-level data: {len(subset)} records")
        
        # Show the latest year’s slice
        latest_year = subset['TimeperiodSortable'].max()
        print(f"   Latest year: {latest_year}")
        display(
            subset[subset['TimeperiodSortable'] == latest_year]
            .sort_values('AreaName')
            .head()
        )
    
    print("\n" + ("="*60))



=== Hypertension QOF prevalence (ID: 219) ===
Columns detected: ['Indicator ID', 'Indicator Name', 'Parent Code', 'Parent Name', 'Area Code', 'Area Name', 'Area Type', 'Sex', 'Age', 'Category Type', 'Category', 'Time period', 'Value', 'Lower CI 95.0 limit', 'Upper CI 95.0 limit', 'Lower CI 99.8 limit', 'Upper CI 99.8 limit', 'Count', 'Denominator', 'Value note', 'Recent Trend', 'Compared to England value or percentiles', 'Compared to percentiles', 'Time period Sortable', 'New data', 'Compared to goal', 'Time period range']


,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,Category,Time period,Value,Lower CI 95.0 limit,Upper CI 95.0 limit,Lower CI 99.8 limit,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to percentiles,Time period Sortable,New data,Compared to goal,Time period range
0,219,Hypertension: QOF prevalence,NaN,NaN,E92000001,England,England,Persons,All ages,NaN,NaN,2009/10,13.351443,13.342443,13.360448,NaN,NaN,7321472.0,54836561.0,NaN,NaN,Not compared,Not compared,20090000,NaN,NaN,1y
1,219,Hypertension: QOF prevalence,NaN,NaN,E92000001,England,England,Persons,All ages,"County & UA deprivation deciles in England (IMD2019, 4/21 geography)",Most deprived decile (IMD2019),2009/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Should be treated with caution, as it contains areas for which values have not been presented due to Nottingham Univ...",NaN,Not compared,Not compared,20090000,NaN,NaN,1y
2,219,Hypertension: QOF prevalence,NaN,NaN,E92000001,England,England,Persons,All ages,"County & UA deprivation deciles in England (IMD2019, 4/21 geography)",Fourth less deprived decile (IMD2019),2009/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Should be treated with caution, as it contains areas for which values have not been presented due to Nottingham Univ...",NaN,Not compared,Not compared,20090000,NaN,NaN,1y
3,219,Hypertension: QOF prevalence,NaN,NaN,E92000001,England,England,Persons,All ages,"CCG deprivation deciles in England (IMD2019, 2021 CCGs)",Most deprived decile,2009/10,12.961620,12.925393,12.997932,12.904542,13.018912,427019.0,3294488.0,Aggregated from all known lower geography values,NaN,Lower 99.8,Not compared,20090000,NaN,NaN,1y
4,219,Hypertension: QOF prevalence,NaN,NaN,E92000001,England,England,Persons,All ages,"CCG deprivation deciles in England (IMD2019, 2021 CCGs)",Second most deprived decile,2009/10,14.465121,14.428924,14.501393,14.408085,14.522345,523653.0,3620108.0,Aggregated from all known lower geography values,NaN,Higher 99.8,Not compared,20090000,NaN,NaN,1y


KeyError: 'Could not find any geography-type column in the DataFrame.'

### 2. Fetch and Inspect Data for Each Profile

We will now loop through our ranked profiles. For each one, we'll get the list of its indicators, pick the first one, and download all the national data for it.

## Conclusion

This direct approach has been successful. By fetching the full national dataset for a sample indicator from each profile, we have confirmed the data structure provided by the API.

We can consistently see the `Area Code`, `Area Name`, and other geographic columns alongside the health data (`Value`, `Count`, etc.).

This validates our strategy for the next step: we can now confidently proceed to download the full datasets for all our target indicators and then use the `Area Code` column to filter for the 7 South West ICBs.